In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver

In [ ]:
from langgraph.graph.message import add_messages

class ChatState(TypedDict):
    
    messages: Annotated[list[BaseMessage], add_messages]
    

In [ ]:
llm = ChatOpenAI()

def chat_node(state: ChatState) -> ChatState:

    # take user input from state
    messages = state["messages"]

    # send to LLM
    response = llm.invoke(messages)

    # get response from LLM
    return {"messages": [response]}

In [ ]:
checkpointer = MemorySaver()
graph = StateGraph(ChatState)

# add nodes
graph.add_node("chat_node", chat_node)

# define edges
graph.add_edge(START, "chat_node")
graph.add_edge("chat_node", END)

chatbot = graph.compile(checkpointer=checkpointer)

In [ ]:
initial_state = {
    "messages": [HumanMessage(content="Hello, how are you?")]
}

chatbot.invoke(initial_state)["messages"][-1].content

In [ ]:
thread_id = '1'
while True:

    user_input = input("You: ")
    print("user_input:", user_input)

    if user_input.strip().lower() in ["exit", "quit", "bye"]:
        break

    config = {'configurable' : {'thread_id': thread_id}}

    response = chatbot.invoke({ "messages": [HumanMessage(content=user_input)]}, config=config)
    print("Bot:", response["messages"][-1].content)
            